In [ ]:
import torch
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
import numpy as np
import pandas as pd
import re
import os
import pprint
from selenium import webdriver
import time
import urllib.request
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## factcheck 데이터 크롤링

여러 기사들과 라벨링된 데이터들을 수집

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
def text_preprocess(text):
    text = re.sub("(<span class='quot[0-9]'>|\n\r\n|</span>|<br/>|<br />|([^0-9가-힣A-Za-z. ]))", "", text)
    return text

content_list = []
fact_list = []
for i in range(1800, 4910):
    try:
        url = f"https://factcheck.snu.ac.kr/facts/show?id={i}"
        driver = webdriver.Chrome('chromedriver')

        try:
            driver.get(url)

            wait = WebDriverWait(driver, 10)
            element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#__next > div > div.jsx-1740931948.layout-inner-container > div > div.jsx-2413405417.page-inner-container > div.jsx-2413405417.fact-show-score-card-continer > div.jsx-2413405417.score-inner-container > div.jsx-883562973 > div > div > div > p:nth-child(1)")))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            list1 = []
            for i in range(1, 10):
                content = text_preprocess(soup.select(f"#__next > div > div.jsx-1740931948.layout-inner-container > div > div.jsx-2413405417.page-inner-container > div.jsx-2413405417.fact-show-score-card-continer > div.jsx-2413405417.score-inner-container > div.jsx-883562973 > div > div > div > p:nth-child({i})")[0].get_text())
                list1.append(content)
                if (i == 2) & (content == ''):
                    continue
                elif (i != 1) & (content == ''):
                    break
            
            list1 = [i.strip() for i in list1]

            list2 = []
            if '검증 대상' in list1:
                list2.append('.'.join(list1[list1.index('검증 대상')+1:]))
            elif '검증대상' in list1:
                list2.append('.'.join(list1[list1.index('검증대상')+1:]))
            else:
                list2.append('검증 대상 없음')
            content_list.append(list2[0])

            label = soup.select('#__next > div > div.jsx-1740931948.layout-inner-container > div > div.jsx-2413405417.page-inner-container > div.jsx-2413405417.fact-show-score-card-continer > div.jsx-2413405417.dial-press-logo-container > div.jsx-2413405417.fact-dial-container')[0].get_text()
            fact_list.append(label)
        except Exception as e:
            print(f"Error occurred for i = {i}: {str(e)}")
            content_list.append('공백')
            fact_list.append('공백')
        
    finally:
        driver.quit()

In [ ]:
print(content_list.count('검증 대상 없음'), fact_list.count('공백'))
print(len(content_list), len(fact_list))

In [ ]:
df = pd.DataFrame({'검증 대상 내용' : content_list, '사실 여부' : fact_list})
df.head(5)

In [ ]:
os.chdir('C:/Users/cts08/Desktop')
df.to_csv('fc_data.csv')

In [ ]:
data = pd.read_csv('C:/Users/cts08/Desktop/fc_data.csv')
data.index = data.index + 1
factcheck_list = data.drop(labels='Unnamed: 0', axis = 1)

In [ ]:
none_list = factcheck_list[factcheck_list['content'] == '검증 대상 없음']

In [ ]:
for i in list(none_list.index):
    try:
        url = f"https://factcheck.snu.ac.kr/facts/show?id={i}"
        driver = webdriver.Chrome('chromedriver')

        try:
            driver.get(url)

            wait = WebDriverWait(driver, 10)
            element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#__next > div > div.jsx-1740931948.layout-inner-container > div > div.jsx-2413405417.page-inner-container > div.jsx-2413405417.fact-show-score-card-continer > div.jsx-2413405417.score-inner-container > div.jsx-883562973 > div > div > div > p:nth-child(1)")))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            content = text_preprocess(soup.select('#__next > div > div.jsx-1740931948.layout-inner-container > div > div.jsx-2413405417.background-container > div > div:nth-child(2) > div.jsx-2413405417.fact-lead-message')[0].get_text())
            none_list.loc[i, 'content'] = content
            
        except Exception as e:
            print(f"Error occurred for i = {i}: {str(e)}")
            none_list.loc[i, 'content'] = '없음'
        
    finally:
        driver.quit()

In [ ]:
f_list = factcheck_list.copy()
f_list.head(3)

In [ ]:
none_list.loc[1, 'content'], f_list.loc[i, 'content']

In [ ]:
for i in list(none_list.index):
    f_list.loc[i,'content'] = none_list.loc[i, 'content']

In [ ]:
os.chdir('C:/Users/cts08/Desktop')
f_list.to_csv('factcheck_data.csv')

### label 변환

In [ ]:
#factcheck data 불러오기
fc_data = pd.read_csv('C:/Users/cts08/Desktop/factcheck_data.csv', encoding = 'utf-8', usecols = [1,2]) # csv 불러오기
df1 = fc_data.copy()

df1 = df1[(df1.fact != '공백') & (df1.fact != '절반의 사실') & (df1.fact != '판단 유보')] # 필요한 binary label 이외 값 제거

df1.loc[(df1['fact'] == '사실') | (df1['fact'] == '대체로 사실'), 'fact'] = 1
df1.loc[(df1['fact'] == '전혀 사실 아님') | (df1['fact'] == '대체로 사실 아님'), 'fact'] = 0 # label 1(True), 0(False)로 변경

#없음 내용 삭제
idx = df1[df1['content'] == "없음"].index
df1.drop(idx , inplace=True)
df1 = df1.dropna(axis=0)

df1 = df1.reset_index(drop=True)
df1.rename(columns={'content':'contents','fact':'label'}, inplace=True)

#crawling data(summary model) 불러오기

label = [1]*len(summary)
df2 = pd.DataFrame({'contents': summary, 'label': label})

final_data = pd.concat([df1, df2])
final_data['label'].value_counts()

In [ ]:
final_data.to_csv('final_data.csv')

## 실제 뉴스 크롤링

fackcheck에서 가짜뉴스 데이터와 올바른뉴스의 분포가 비대칭이라고 판단하여 올바른 데이터셋을 더 추가해주는 작업

#### 중앙일보(450개)

In [ ]:
def text_preprocess(text):
    text = re.sub("(<span class='quot[0-9]'>|\n\r\n|</span>|<br/>|<br />|([^0-9가-힣A-Za-z. ]))","",text)
    return text

In [ ]:
joongang_content = []
count = 0
while count < 450:
    num = 25173910 - count*10
    joongang_url = 'https://www.joongang.co.kr/article/' + str(num)
    requested = requests.get(joongang_url, 'html.parser')
    soup = BeautifulSoup(requested.content)

    news_content = soup.select('#article_body') #textBody

    if news_content != []:
        joongang_content.append(text_preprocess(news_content[0].get_text()))
        count += 1
    elif news_content == []:
        count += 1

#### 서울일보(1200개)

In [ ]:
seoul_content = []
count = 0
while count < 1200:
    num = 619000 - count*10
    joongang_url = 'http://www.seoulilbo.com/news/articleView.html?idxno=' + str(num)
    requested = requests.get(joongang_url, headers={'User-Agent':'Mozilla/5.0'})
    soup = BeautifulSoup(requested.content)

    news_content = soup.select('#article-view-content-div') #textBody

    if news_content != []:
        seoul_content.append(text_preprocess(news_content[0].get_text()))
        count += 1
    elif news_content == []:
        count += 1

#### 내일신문(1200개)

In [ ]:
naeil_content = []
count = 0
while count < 1200:
    num = 465656 - count*10
    naeil_url = 'http://www.naeil.com/news_view/?id_art=' + str(num)
    requested = requests.get(naeil_url, headers={'User-Agent':'Mozilla/5.0'})
    soup = BeautifulSoup(requested.content)

    news_content = soup.select('#contents') #textBody

    if news_content != []:
        naeil_content.append(text_preprocess(news_content[0].get_text()))
        count += 1
    elif news_content == []:
        count += 1

#### 뉴스 통합

In [ ]:
news_content = joongang_content + seoul_content + naeil_content

In [ ]:
label = [1]*len(news_content)
len(label)

In [ ]:
true_df = pd.DataFrame(list(zip(news_content,label)), columns = ['contents','label'])
true_df.head(5)

In [ ]:
# 요약 모델
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model_sm = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

summary = []

for i in true_df['contents']:
    content = i
    text = re.sub(' +', ' ', content)
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

    if len(input_ids) > 1024: continue
    summary_ids = model_sm.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
    summary_text = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    summary.append(summary_text)

In [ ]:
true_df.to_csv('true_data.csv')

### test data 생성

In [ ]:
seoul_content = []
count = 0
while count < 150:
    num = 619000 - count*100
    seoul_url = 'http://www.seoulilbo.com/news/articleView.html?idxno=' + str(num)
    requested = requests.get(seoul_url, 'html.parser')
    soup = BeautifulSoup(requested.content)

    news_content = soup.select('#article-view-content-div') #textBody

    if news_content != []:
        seoul_content.append(text_preprocess(news_content[0].get_text()))
        count += 1
    elif news_content == []:
        count += 1

In [ ]:
test_label = [1]*len(seoul_content)
len(test_label)

In [ ]:
test_label = [1]*len(seoul_content)
len(test_label)
test_df = pd.DataFrame(list(zip(seoul_content,test_label)), columns = ['contents','label'])
test_df.head(5)

In [ ]:
# 요약 모델
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model_sm = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

summary = []

for i in df['contents']:
    content = i
    text = re.sub(' +', ' ', content)
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

    if len(input_ids) > 1024: continue
    summary_ids = model_sm.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
    summary_text = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    summary.append(summary_text)

In [ ]:
label = [1]*len(summary)
df2 = pd.DataFrame({'contents': summary, 'label': label})

df2.to_csv('test.csv')